In [3]:
import pandas as pd
from transformers import AutoModelForSequenceClassification
from transformers import TrainingArguments
from transformers import AutoTokenizer
import numpy as np
from datasets import load_metric
from transformers import Trainer

In [4]:
dataset = pd.read_csv('cleaned_fake_news_stories.csv', index_col=0)

In [22]:
dataset = load_dataset('csv', data_files='cleaned_fake_news_stories.csv')

Using custom data configuration default-dbbb3976730fb34d
Reusing dataset csv (C:\Users\laram\.cache\huggingface\datasets\csv\default-dbbb3976730fb34d\0.0.0\433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519)


  0%|          | 0/1 [00:00<?, ?it/s]

In [25]:
dataset

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'article_content', 'fake_or_satire'],
        num_rows: 220
    })
})

In [6]:
shuffled_dataset = dataset.sample(frac=1)

In [7]:
shuffled_dataset[]

,article_content,fake_or_satire
141,Report finds Firefighters at risk due to expos...,Satire
79,"With the exceptions of capitalism, God, and lo...",Fake
58,The American Public is With You!\n\n–Not.\n\nP...,Fake
112,WASHINGTON — In a move that threatened to upse...,Satire
66,Scary Roblox Games to Play with Friends is a f...,Fake
...,...,...
161,A man called police reporting he was called a ...,Satire
196,(SP) – Protesters gathered outside NFL headqua...,Satire
140,"As Christmas season continues in full swing, m...",Satire
42,"Simple, secure purchase & transfer Once paymen...",Fake


In [8]:
train_dataset = shuffled_dataset[:176]
test_dataset = shuffled_dataset[176:]

In [17]:
load_dataset(train_dataset)

AttributeError: 'DataFrame' object has no attribute 'split'

In [16]:
Dataset({})

NameError: name 'Dataset' is not defined

In [15]:
train_dataset.astype('Dataset')

TypeError: data type 'Dataset' not understood

In [11]:
model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=2)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

In [13]:
for index, row in train_dataset.iterrows():
    print(row["article_content"])

Report finds Firefighters at risk due to exposure to the chemical Dihydrogen Monoxide

It seems everyday there is a new risk to being a firefighter. Chemicals found on fire scenes and absorbed into turnout gear have been linked to many illness. Now a chemical found in fire hose is raising an alarm.

A newly published scientific study in the Journal of Chemical Reaction and Human Hazards in Extreme Workplace Environments, sheds light on this new chemical danger.

According to the study…

Dihydrogen Monoxide is a colorless and odorless chemical compound. At room temperatures it is a non-viscous liquid, but at high temperatures as found in fires it can turn into a vapor.

Inhalation of Dihydrogen Monoxide can cause respiratory distress. Death will result from an excessive amount being inhaled.

Exposure to the vapor from Dihydrogen Monoxide will cause burns. Again death is a possibility with an excessive exposure.

Fire hoses release Dihydrogen Monoxide during normal firefighting operatio

In [8]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

def tokenize_dataset(dataset,tokenizer):
    tokenized_dataset = []
    for item in dataset:
        tokenized = tokenizer(item["article_content"],padding="max_length", truncation=True)
        item.update(tokenized)
        tokenized_dataset.append(item)
    return tokenized_dataset

tokenized_train = tokenize_dataset(train_dataset,tokenizer)
tokenized_test = tokenize_dataset(test_dataset,tokenizer)

In [10]:
training_args = TrainingArguments("test_trainer",evaluation_strategy="epoch")

In [12]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    metric = load_metric("accuracy")
    return metric.compute(predictions=predictions, references=labels)

In [14]:
trainer = Trainer(
    model=model, 
    args=training_args, 
    train_dataset=tokenized_train, 
    #eval_dataset=tokenized_test,
    compute_metrics = compute_metrics
)

In [15]:
trainer.train()

***** Running training *****
  Num examples = 2000
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 750


Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 

In [16]:
trainer.evaluate(tokenized_test)

***** Running Evaluation *****
  Num examples = 1000
  Batch size = 8


KeyboardInterrupt: 

In [ ]:
predictions = trainer.predict(tokenized_test)

In [ ]:
def print_predictions_by_type(pred_type, predictions):
    print('='*200)
    print(pred_type)
    for pred in predictions:
        print(pred)

def get_predictions_by_type(dataset):
    for i,item in enumerate(dataset):
        predicted = np.argmax(predictions.predictions[i])
        gold = item['label']
        if predicted == gold: # prediction is correct
            if predicted == 1:
                print('TP: ' + item['text'])
            else:
                print('TN: ' + item['text'])
        else: # prediction is incorrect
            if predicted == 1:
                print('FP: ' + item['text'])
            else:
                print('FN: ' + item['text'])


get_predictions_by_type(test_dataset)